In [1]:
from dotenv import load_dotenv

import openai
import os
load_dotenv()

import logging
import sys
import os

import numexpr as ne

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
openai.api_key = os.getenv('OPENAI_API_KEY')


os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'

In [2]:
from llama_index import ServiceContext, VectorStoreIndex
from llama_index import SimpleDirectoryReader
from llama_index.llms import OpenAI
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor


documents = SimpleDirectoryReader('../assets/ebook/').load_data()

In [3]:
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [4]:
from llama_index.node_parser import SimpleNodeParser

In [5]:
simple_parser = SimpleNodeParser.from_defaults()

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-mpnet-base-v2",
    max_length=512
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [6]:
ctx = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model
)


nodes = node_parser.get_nodes_from_documents(documents)
sentence_index = VectorStoreIndex(nodes, service_context=ctx)

In [7]:
query_engine = sentence_index.as_query_engine(
    similarity_top_k=2,
    node_postprocessors = [
        MetadataReplacementPostProcessor(
            target_metadata_key="window"
        )
    ]
)

In [8]:
window_response = query_engine.query(
    "Bệnh động kinh là gì?"
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [9]:
print(window_response)

Bệnh động kinh là một căn bệnh mà người bệnh gặp phải cơn co giật không kiểm soát được. Có thể điều trị bệnh động kinh và có thể chữa khỏi hoàn toàn trong một số trường hợp. Tuy nhiên, không phải tất cả người bệnh đều có thể khỏi hoàn toàn, nhưng vẫn có thể sử dụng thuốc để duy trì và đảm bảo một cuộc sống khỏe mạnh.


In [12]:
for i in window_response.source_nodes:
    print(i.text)
    print(i.score)

Bạn chỉ nên nghiêng đầu người bệnh sang một bên, giúp họ nới lỏng quần áo và giữ người bệnh ở một tư thế thoải mái nhất.  Cần phải liên tục để ý tới người bệnh trong cơn co giật.<h2><strong>Điều trị bệnh động kinh tại BVĐK Tâm Anh</strong></h2> Nhiều nghiên cứu đã ghi nhận những khó khăn và bất lợi mà các trường hợp mắc bệnh động kinh đang gặp phải, phần lớn là do quan điểm sai lầm, thái độ không đúng của cả cộng đồng về căn bệnh này.  Chính vì thế, Tổ chức Y tế Thế giới (WHO) đã phát động chiến dịch toàn cầu với mục tiêu chống lại động kinh, đồng thời cải thiện chất lượng cuộc sống cho những người mắc bệnh.  PGS.TS.BSCKII Nguyễn Văn Liệu &#8211; chuyên gia đầu ngành về Thần kinh cũng khẳng định: “<i>Bệnh động kinh là bệnh cần chữa và có thể chữa khỏi hoàn toàn.  Tỉ lệ chữa khỏi bệnh khá cao, có thể lên tới 70%.  Nghĩa là khoảng 70% người bệnh có thể cắt được cơn động kinh và trở lại cuộc sống bình thường sau một quá trình điều trị từ 2,5 &#8211; 3 năm. 
0.6475218169644312
PGS.TS.BSCKI